In [2]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from base import supres
from exputils import *

C:\Users\david.vesely\AppData\Local\conda\conda\envs\crypto36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Load data

In [5]:
dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
# dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['change_1', 'change_2', 'change_3']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

data_dict = {p: read_all(p, dropbox_dir) for p in pairs}
train = read_tsv('train.tsv')
validation = read_tsv('validation.tsv')
test = read_tsv('test.tsv')

# Train validation split

In [33]:
from itertools import product

def find_support(i, supports):
    selected = supports[supports['index'] <= i]['support']
    return None if len(selected) == 0 else selected.iloc[-1]


def ohlc_features_f(args):
    data_dict, supports_dict, pair, window_size = args
    supports = supports_dict[pair][window_size]
    data = data_dict[pair]
    s = [find_support(i, supports) for i in range(len(data))]
    return pair, window_size, s


def calc_ohlc_features(data_dict, supports_dict, output_file):
    args = product([data_dict], [supports_dict], pairs, window_sizes)
    results = run_parallel(ohlc_supports_f, args)
    supports_for_data_dict = dict_from_list(results)
    save_model(supports_for_data_dict, output_file)

In [34]:
supports_dict_train = load_model('supports_train.pkl')
calc_ohlc_supports(train_dict, supports_dict_train, 'ohlc_supports_train.pkl')